<a href="https://colab.research.google.com/github/fabianoborgesbsb/APC_study/blob/main/APC_study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import pandas_gbq
import matplotlib.pyplot as plt

In [ ]:
import os

file_path = '/content/drive/My Drive/APC_study.ipynb'
print(os.path.exists(file_path))  # Deve retornar: True


In [ ]:
import nbformat

with open(file_path, 'r', encoding='utf-8') as f:
    nb = nbformat.read(f, as_version=4)

for cell in nb.cells:
    if 'metadata' in cell and 'widgets' in cell['metadata']:
        del cell['metadata']['widgets']

# Salvar como novo notebook (opcional)
with open('/content/drive/My Drive/APC_study_fixed.ipynb', 'w', encoding='utf-8') as f:
    nbformat.write(nb, f)



# Average APC each country

In [ ]:
%%bigquery --project=insyspo

SELECT countries.country_code,
  ANY_VALUE(countries.country) AS country,
  AVG(CAST(apcs.value_usd AS FLOAT64)) AS avg_value_usd
FROM `insyspo.publicdb_openalex_2024_04_rm.works_apc` AS apcs
JOIN (
  SELECT DISTINCT apcs.id, authorships.institution_id
  FROM `insyspo.publicdb_openalex_2024_04_rm.works_apc` AS apcs
  JOIN `insyspo.publicdb_openalex_2024_04_rm.works_authorships` AS authorships
  ON apcs.id = authorships.work_id
) AS institutions
ON apcs.id = institutions.id
JOIN `insyspo.publicdb_openalex_2024_04_rm.institutions` AS countries
ON institutions.institution_id = countries.id
GROUP BY countries.country_code
ORDER BY avg_value_usd DESC;

# Average APC every year each country

In [ ]:
%%bigquery --project=insyspo

DROP TABLE IF EXISTS userdb_luis_fabiano_borges.apcs;
CREATE TABLE userdb_luis_fabiano_borges.apcs AS (
  SELECT countries.country_code,
    institutions.publication_year,
    ANY_VALUE(countries.country) AS country,
    AVG(CAST(apcs.value_usd AS FLOAT64)) AS avg_value_usd,
    COUNT(DISTINCT apcs.id) AS works
  FROM `insyspo.publicdb_openalex_2024_04_rm.works_apc` AS apcs
  JOIN (
    SELECT DISTINCT apcs.id, authorships.institution_id, works.publication_year
    FROM `insyspo.publicdb_openalex_2024_04_rm.works_apc` AS apcs
    JOIN `insyspo.publicdb_openalex_2024_04_rm.works_authorships` AS authorships
    ON apcs.id = authorships.work_id
    JOIN `insyspo.publicdb_openalex_2024_04_rm.works` AS works
    ON works.id = apcs.id
  ) AS institutions
  ON apcs.id = institutions.id
  JOIN `insyspo.publicdb_openalex_2024_04_rm.institutions` AS countries
  ON institutions.institution_id = countries.id
  WHERE institutions.publication_year >= 1990
    AND institutions.publication_year <= 2024
  GROUP BY countries.country_code, institutions.publication_year
  HAVING AVG(CAST(apcs.value_usd AS FLOAT64)) > 0.0
  ORDER BY institutions.publication_year, countries.country_code
)

In [ ]:
%%bigquery df --project=insyspo

SELECT *
FROM userdb_luis_fabiano_borges.apcs

# Each field

In [ ]:
%%bigquery --project=insyspo

DROP TABLE IF EXISTS userdb_luis_fabiano_borges.apcs_engineering;
CREATE TABLE userdb_luis_fabiano_borges.apcs_engineering AS (
  SELECT countries.country_code,
    institutions.publication_year,
    ANY_VALUE(countries.country) AS country,
    AVG(CAST(apcs.value_usd AS FLOAT64)) AS avg_value_usd,
    COUNT(DISTINCT apcs.id) AS works
  FROM `insyspo.publicdb_openalex_2024_04_rm.works_apc` AS apcs
  JOIN (
    SELECT DISTINCT apcs.id, authorships.institution_id,
      works.publication_year
    FROM `insyspo.publicdb_openalex_2024_04_rm.works_apc` AS apcs
    JOIN `insyspo.publicdb_openalex_2024_04_rm.works_authorships` AS authorships
    ON apcs.id = authorships.work_id
    JOIN `insyspo.publicdb_openalex_2024_04_rm.works` AS works
    ON works.id = apcs.id
    JOIN `insyspo.publicdb_openalex_2024_04_rm.works_topics` AS works_topics
    ON works.id = works_topics.work_id
    JOIN `insyspo.publicdb_openalex_2024_04_rm.topics` AS topics
    ON works_topics.topic_id = topics.id
    WHERE topics.field = 22
  ) AS institutions
  ON apcs.id = institutions.id
  JOIN `insyspo.publicdb_openalex_2024_04_rm.institutions` AS countries
  ON institutions.institution_id = countries.id
  WHERE institutions.publication_year >= 1990
  AND institutions.publication_year <= 2024
  GROUP BY countries.country_code, institutions.publication_year
  HAVING AVG(CAST(apcs.value_usd AS FLOAT64)) > 0.0
  ORDER BY institutions.publication_year, countries.country_code
);

# Correct tables for missing country codes

Aggregate

In [ ]:
%%bigquery df --project=insyspo

SELECT *
FROM userdb_luis_fabiano_borges.apcs

In [ ]:
df.columns

In [ ]:
row_drop = []
for i, cc in enumerate(df['country_code']):
  if cc == None:
    if df.loc[i,'country'] == 'France':
      ccode = 'FR'
    elif df.loc[i,'country'] == 'India':
      ccode = 'IN'
    elif df.loc[i,'country'][:6] == 'United':
      ccode = 'US'
    elif df.loc[i,'country'] == 'China':
      ccode = 'CN'
    elif df.loc[i,'country'] == 'Netherlands':
      ccode = 'NL'

    k = df.index[(df['publication_year'] == df.loc[i,'publication_year']) & (df['country_code'] == ccode)][0]
    df.loc[k,'avg_value_usd'] = (df.loc[k,'avg_value_usd']*df.loc[k,'works'] +
                                 df.loc[i,'avg_value_usd']*df.loc[i,'works']) / (df.loc[k,'works'] + df.loc[i,'works'])
    row_drop.append(i)

In [ ]:
df['country'].replace(["The Netherlands"], ["Netherlands"], inplace=True)
df[df['country']=='Netherlands']

In [ ]:
df.drop(row_drop,axis=0, inplace=True)

In [ ]:
df.to_gbq(project_id='insyspo',destination_table='userdb_luis_fabiano_borges.apcs',if_exists ='replace')

Computer Science

In [ ]:
%%bigquery df --project=insyspo

SELECT *
FROM userdb_luis_fabiano_borges.apcs_computer_science

In [ ]:
row_drop = []
for i, cc in enumerate(df['country_code']):
  if cc == None:
    if df.loc[i,'country'] == 'France':
      ccode = 'FR'
    elif df.loc[i,'country'] == 'India':
      ccode = 'IN'
    elif df.loc[i,'country'][:6] == 'United':
      ccode = 'US'
    elif df.loc[i,'country'] == 'China':
      ccode = 'CN'
    elif df.loc[i,'country'] == 'Netherlands':
      ccode = 'NL'

    k = df.index[(df['publication_year'] == df.loc[i,'publication_year']) & (df['country_code'] == ccode)][0]
    df.loc[k,'avg_value_usd'] = (df.loc[k,'avg_value_usd']*df.loc[k,'works'] +
                                 df.loc[i,'avg_value_usd']*df.loc[i,'works']) / (df.loc[k,'works'] + df.loc[i,'works'])
    row_drop.append(i)

df['country'].replace(["The Netherlands"], ["Netherlands"], inplace=True)
df.drop(row_drop,axis=0, inplace=True)
df.to_gbq(project_id='insyspo',destination_table='userdb_luis_fabiano_borges.apcs_computer_science',if_exists ='replace')


Medicine


In [ ]:
%%bigquery df --project=insyspo

SELECT *
FROM userdb_luis_fabiano_borges.apcs_medicine

In [ ]:
row_drop = []
for i, cc in enumerate(df['country_code']):
  if cc == None:
    if df.loc[i,'country'] == 'France':
      ccode = 'FR'
    elif df.loc[i,'country'] == 'India':
      ccode = 'IN'
    elif df.loc[i,'country'][:6] == 'United':
      ccode = 'US'
    elif df.loc[i,'country'] == 'China':
      ccode = 'CN'
    elif df.loc[i,'country'] == 'Netherlands':
      ccode = 'NL'

    k = df.index[(df['publication_year'] == df.loc[i,'publication_year']) & (df['country_code'] == ccode)][0]
    df.loc[k,'avg_value_usd'] = (df.loc[k,'avg_value_usd']*df.loc[k,'works'] +
                                 df.loc[i,'avg_value_usd']*df.loc[i,'works']) / (df.loc[k,'works'] + df.loc[i,'works'])
    row_drop.append(i)

df['country'].replace(["The Netherlands"], ["Netherlands"], inplace=True)
df.drop(row_drop,axis=0, inplace=True)
df.to_gbq(project_id='insyspo',destination_table='userdb_luis_fabiano_borges.apcs_medicine',if_exists ='replace')


Engineering

In [ ]:
%%bigquery df --project=insyspo

SELECT *
FROM userdb_luis_fabiano_borges.apcs_engineering

In [ ]:
row_drop = []
for i, cc in enumerate(df['country_code']):
  if cc == None:
    if df.loc[i,'country'] == 'France':
      ccode = 'FR'
    elif df.loc[i,'country'] == 'India':
      ccode = 'IN'
    elif df.loc[i,'country'][:6] == 'United':
      ccode = 'US'
    elif df.loc[i,'country'] == 'China':
      ccode = 'CN'
    elif df.loc[i,'country'] == 'Netherlands':
      ccode = 'NL'

    k = df.index[(df['publication_year'] == df.loc[i,'publication_year']) & (df['country_code'] == ccode)][0]
    df.loc[k,'avg_value_usd'] = (df.loc[k,'avg_value_usd']*df.loc[k,'works'] +
                                 df.loc[i,'avg_value_usd']*df.loc[i,'works']) / (df.loc[k,'works'] + df.loc[i,'works'])
    row_drop.append(i)

df['country'].replace(["The Netherlands"], ["Netherlands"], inplace=True)
df.drop(row_drop,axis=0, inplace=True)
df.to_gbq(project_id='insyspo',destination_table='userdb_luis_fabiano_borges.apcs_engineering',if_exists ='replace')


Physics

In [ ]:
%%bigquery df --project=insyspo

SELECT *
FROM userdb_luis_fabiano_borges.apcs_physics

In [ ]:
row_drop = []
for i, cc in enumerate(df['country_code']):
  if cc == None:
    if df.loc[i,'country'] == 'France':
      ccode = 'FR'
    elif df.loc[i,'country'] == 'India':
      ccode = 'IN'
    elif df.loc[i,'country'][:6] == 'United':
      ccode = 'US'
    elif df.loc[i,'country'] == 'China':
      ccode = 'CN'
    elif df.loc[i,'country'] == 'Netherlands':
      ccode = 'NL'

    k = df.index[(df['publication_year'] == df.loc[i,'publication_year']) & (df['country_code'] == ccode)][0]
    df.loc[k,'avg_value_usd'] = (df.loc[k,'avg_value_usd']*df.loc[k,'works'] +
                                 df.loc[i,'avg_value_usd']*df.loc[i,'works']) / (df.loc[k,'works'] + df.loc[i,'works'])
    row_drop.append(i)

df['country'].replace(["The Netherlands"], ["Netherlands"], inplace=True)
df.drop(row_drop,axis=0, inplace=True)
df.to_gbq(project_id='insyspo',destination_table='userdb_luis_fabiano_borges.apcs_physics',if_exists ='replace')
